# Load SARS-CoV-2 Strain Variant Data from CNCB
**[Work in progress]**

This notebook downloads and standardizes viral strain variation data from CNCB for ingestion into a Knowledge Graph.

Data source: [China National Center for Bioinformation, 2019 Novel Coronavirus Resource (2019nCoVR)](https://bigd.big.ac.cn/ncov/release_genome)

Author: Peter Rose (pwrose@ucsd.edu)

In [1]:
import os
import shutil
import glob
import ftplib
import re
import dateutil
import time
import pandas as pd
from pathlib import Path

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [3]:
# Path will take care of handling operating system differences.
NEO4J_IMPORT = Path(os.getenv('NEO4J_IMPORT'))
print(NEO4J_IMPORT)

/Users/peter/Library/Application Support/com.Neo4j.Relate/data/dbmss/dbms-8bf637fc-0d20-4d9f-9c6f-f7e72e92a4da/import


In [4]:
# Create a directory to cache variation data that could not be parsed
CACHE_FAILED = Path(NEO4J_IMPORT / 'cache/failed/cncb')
CACHE_FAILED.mkdir(parents=True, exist_ok=True)

In [5]:
# Create a directory to cache raw variation data gff3 files
CACHE_RAW_CNCB = Path(NEO4J_IMPORT / 'cache/raw/cncb')
CACHE_RAW_CNCB.mkdir(parents=True, exist_ok=True)

In [6]:
# Create a directory to cache processed variation data csv files
CACHE_PROCESSED_CNCB = Path(NEO4J_IMPORT / 'cache/processed/cncb')
CACHE_PROCESSED_CNCB.mkdir(parents=True, exist_ok=True)

## Download SARS-CoV-2 Variation Data

Create a list of local and remote directories

In [7]:
# directories on the FTP server
remote_dirs = []
# local directories to cache raw data files
raw_dirs = []
# local directories to cache processed data files
proc_dirs = []

# subdirectories are named: a ... n (note, this may change)
subdirs = [chr(x) for x in range(ord('a'), ord('n') + 1)]

Setup local cache directories if they don't exit

In [8]:
for subdir in raw_dirs:
    subdir.mkdir(exist_ok=True)
for subdir in proc_dirs:   
    subdir.mkdir(exist_ok=True)

In [9]:
t1 = time.time()

In [10]:
def parse_strain_id(filename):
    # parse strain id from file name, e.g. ..cache/raw/cncb/a/2019-nCoV_EPI_ISL_484968_variants.gff3 -> EPI_ISL_484968
    return Path(filename).stem[10:-9]

In [11]:
def parse_base_path(filename):
    # parse base path from file name, e.g. ..cache/raw/cncb/n/2019-nCoV_EPI_ISL_484968_variants.gff3 -> /n/2019-nCoV_EPI_ISL_484968_variants.
    return re.split('cncb', filename)[1][1:-5]

Create a dataframe of raw files and strain identifiers

In [12]:
raw_files = glob.glob(f'{CACHE_RAW_CNCB}/*/*.gff3')
raw_files_df = pd.DataFrame(raw_files, columns=['raw_filename'])
raw_files_df['id'] = raw_files_df['raw_filename'].apply(parse_strain_id)
print("Cached raw gff3 files:", raw_files_df.shape[0])

Cached raw gff3 files: 6003


In [13]:
t2 = time.time()
print("Time to create list of raw files:", t2-t1)

Time to create list of raw files: 0.25726890563964844


Create a dataframe of processed files and strain identifiers

In [14]:
proc_files = glob.glob(f'{CACHE_PROCESSED_CNCB}/*/*.csv')
proc_files_df = pd.DataFrame(proc_files, columns=['proc_filename'])
proc_files_df['id'] = proc_files_df['proc_filename'].apply(parse_strain_id)
print("Cached processed csv files:", proc_files_df.shape[0])

Cached processed csv files: 5993


In [15]:
t3 = time.time()
print("Time to create list of processed files:", t3-t2)

Time to create list of processed files: 0.22624683380126953


In [16]:
unproc_files_df = pd.merge(raw_files_df, proc_files_df, on='id', how='outer', indicator=True).query('_merge=="left_only"')

In [17]:
print("Files to be processed:", unproc_files_df.shape[0])

Files to be processed: 10


In [18]:
names = ['taxon1', 'variantType', 'name', 'start', 'end','x1', 'x2', 'x3','taxon2', 'x4', 'ref', 'alt', 'vepAnnotation']

In [19]:
def split_vep(record):
    # split variant effect predictor record
    items = record.split(',')
    num_items = len(items)
    # example: ['intergenic_variant']
    if num_items == 1:
        return items[0] + ',,,'
    # example: ['missense_variant', 'QHD43415.1:p.1036D>E', 'gene-orf1ab:c.3108gaC>gaA']
    elif num_items == 3:
        return items[0] + ',' + items[1] + ',' + items[2] + ','
    # example: ['upstream_gene_variant', 'DISTANCE=25', 'QHD43415.1', 'gene-orf1ab']     
    elif num_items == 4:
        return items[0] + ',' +  items[2] + ',' + items[3] + ',' + items[1]
    else:
        return ',,,'

In [20]:
def parse_gff3(raw_file):
    filename =  parse_base_path(raw_file)  + '.csv'
    #print('parsing:', raw_file, filename)
    gff3 = pd.read_csv(raw_file, header=None, comment='#', sep='[\t;]', engine='python', names=names)
    try:
        gff3['vepAnnotation'] = gff3['vepAnnotation'].str.replace('VEP=','')
        gff3['vepAnnotation'] = gff3['vepAnnotation'].apply(split_vep)
        gff3[['variantConsequence','proteinVariant','geneVariant', 'distance']] = gff3['vepAnnotation'].str.split(',', expand=True)
        gff3['geneVariant'] = gff3['geneVariant'].str.replace('gene-','')
        gff3['distance'] = gff3['distance'].str.replace('DISTANCE=', '')
        gff3['ref'] = gff3['ref'].str.replace('REF=','')
        gff3['alt'] = gff3['alt'].str.replace('ALT=','')
        gff3 = gff3[['name', 'variantType', 'start', 'end', 'ref', 'alt', 'variantConsequence', 'proteinVariant', 'geneVariant', 'distance']]
        gff3['accession'] = parse_strain_id(raw_file)

        gff3.to_csv(CACHE_PROCESSED_CNCB / filename, index=False)
        return True
    except:
        # skip over files that have no mutations
        valid = False
        with open(raw_file) as fp: 
            while True: 
                line = fp.readline()
                if not line:
                    break
                if line.startswith('##There is no mutation found'):
                    print('No mutations in   :', filename)
                    valid = True
                    break
   
        if not valid:
            print('Parsing failed for:', filename)
            # cache files that failed to be parsed for manual inspection
            shutil.copy(raw_file, CACHE_FAILED)
            return False
        else:
            return True

In [21]:
status = raw_files_df['raw_filename'].apply(parse_gff3)

Parsing failed for: a/2019-nCoV_EPI_ISL_424491_variants.csv
No mutations in   : a/2019-nCoV_EPI_ISL_424471_variants.csv
Parsing failed for: a/2019-nCoV_EPI_ISL_418635_variants.csv
No mutations in   : a/2019-nCoV_EPI_ISL_424538_variants.csv
No mutations in   : a/2019-nCoV_EPI_ISL_417602_variants.csv
Parsing failed for: a/2019-nCoV_EPI_ISL_423003_variants.csv
Parsing failed for: a/2019-nCoV_EPI_ISL_420580_variants.csv
Parsing failed for: a/2019-nCoV_EPI_ISL_415583_variants.csv
No mutations in   : a/2019-nCoV_EPI_ISL_417591_variants.csv
Parsing failed for: a/2019-nCoV_EPI_ISL_420534_variants.csv


In [22]:
t4 = time.time()
print("Time to preprocess new files:", t4-t3)

Time to preprocess new files: 66.72967624664307


#### Standardize node property names (CURIEs and URIs)

In [23]:
# https://registry.identifiers.org/registry/insdc
insdc_pattern = re.compile('^([A-Z]\d{5}|[A-Z]{2}\d{6}|[A-Z]{4}\d{8}|[A-J][A-Z]{2}\d{5})(\.\d+)?$')
# https://registry.identifiers.org/registry/refseq
refseq_pattern = re.compile('^(((AC|AP|NC|NG|NM|NP|NR|NT|NW|XM|XP|XR|YP|ZP)_\d+)|(NZ\_[A-Z]{2,4}\d+))(\.\d+)?$')

In [24]:
def assign_curie(id):
    id = id.strip()
    # remove underscore to enable CURIE matching of NCBI reference sequences NC_...
    #id = id.replace('NC_', 'NC') 
    if len(id) > 0:
        if id.startswith('EPI'):
            return 'https://www.gisaid.org/' + id
        elif refseq_pattern.match(id) != None:
            return 'refseq:' + id
        elif insdc_pattern.match(id) != None:
            return 'insdc:' + id
        else:
            # TODO are URIs available for these cases?
            return id
    else:
        return id

In [25]:
# use all processed data files
path = str(CACHE_PROCESSED_CNCB / '*/*.csv')
filenames = glob.glob(path)

variations = pd.concat((pd.read_csv(f, index_col=None, header=0) for f in filenames), ignore_index=True)
variations.fillna('', inplace=True)

print('Number of cached files loaded:',len(filenames))

Number of cached files loaded: 5993


In [26]:
t5 = time.time()
print('Time to load cached files:', t5-t4)

Time to load cached files: 20.383455991744995


List of variant types and consequences:

https://uswest.ensembl.org/info/genome/variation/prediction/classification.html

https://uswest.ensembl.org/info/genome/variation/prediction/predicted_data.html#consequences

#### Extract protein position and protein id from proteinVariant string

Example: QHD43415.1:p.5828P>L

proteinPosition: 5828
proteinId: QHD43415

In [27]:
position_pattern = re.compile(':p\.(.*?)[A-Z|\-]+')

In [28]:
def extract_protein_position(s):
    if s == '':
        return s
    else:
        groups = position_pattern.search(s)
        if groups == None:
            return ''
        else:
            return groups.group(1)

In [29]:
variations['proteinPosition'] = variations['proteinVariant'].apply(extract_protein_position)

In [30]:
variations['proteinAccession'] = variations['proteinVariant'].apply(lambda s: s.split('.')[0] if '.' in s else '')

In [31]:
variations['proteinAccession'].unique()

array(['', 'QHD43415', 'QHD43416', 'QHD43417', 'QHI42199', 'QHD43418',
       'QHD43419', 'QHD43423', 'QHD43421', 'QHD43422', 'QHD43420'],
      dtype=object)

#### Assign SARS-CoV-2 taxonomy id

In [32]:
variations['taxonomyId'] = 'taxonomy:2697049'

#### Assign Reference genome

The first SARS-CoV-2 genome sequence is the reference for the variant annotation below.

[Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1](https://www.ncbi.nlm.nih.gov/nuccore/MN908947)

In [33]:
variations['referenceGenome'] = 'insdc:MN908947' # same as NCBI reference sequence NC_045512

In [34]:
variations['proteinAccession'] = variations['proteinAccession'].apply(lambda s: 'ncbiprotein:' + s if s != '' else s)

In [35]:
variations['accession'] = variations['accession'].apply(assign_curie)

Fix a misspelled terms

In [36]:
variations['variantConsequence'] = variations['variantConsequence'].replace('intergenic_variant||intergenic_variant', 'intergenic_variant')

In [37]:
variations['variantConsequence'] = variations['variantConsequence'].replace('intergenic_variant||intergenic_varia', 'intergenic_variant')

In [38]:
print('variantType:', variations['variantType'].unique())

variantType: ['Deletion' 'SNP' 'Insertion' 'Indel']


In [39]:
print("variantConsequence:", variations['variantConsequence'].unique())

variantConsequence: ['intergenic_variant' 'upstream_gene_variant' 'missense_variant'
 'synonymous_variant' 'inframe_deletion' 'downstream_gene_variant'
 'coding_sequence_variant' 'stop_gained' 'frameshift_variant'
 'protein_altering_variant' 'inframe_insertion' 'stop_lost' 'start_lost']


In [40]:
variations['id'] = variations['referenceGenome'] + ':' + variations['start'].map(str) + '-' + variations['end'].map(str) + '-' + variations['ref'] + '-' + variations['alt']

In [41]:
variations.head()

name variantType  start  end       ref alt  \
0  hCoV-19/USA/WA-UW-1446/2020    Deletion      1    7  AATTAAAG   -   
1  hCoV-19/USA/WA-UW-1446/2020         SNP     34   34         A   T   
2  hCoV-19/USA/WA-UW-1446/2020         SNP     35   35         A   T   
3  hCoV-19/USA/WA-UW-1446/2020         SNP     36   36         C   T   
4  hCoV-19/USA/WA-UW-1446/2020         SNP     37   37         C   A   

   variantConsequence proteinVariant geneVariant distance  \
0  intergenic_variant                                       
1  intergenic_variant                                       
2  intergenic_variant                                       
3  intergenic_variant                                       
4  intergenic_variant                                       

                               accession proteinPosition proteinAccession  \
0  https://www.gisaid.org/EPI_ISL_423010                                    
1  https://www.gisaid.org/EPI_ISL_423010                                    
2  https://www.gisaid.org/EPI_ISL_423010                                    
3  https://www.gisaid.org/EPI_ISL_423010                                    
4  https://www.gisaid.org/EPI_ISL_423010                                    

         taxonomyId referenceGenome                             id  
0  taxonomy:2697049  insdc:MN908947  insdc:MN908947:1-7-AATTAAAG--  
1  taxonomy:2697049  insdc:MN908947       insdc:MN908947:34-34-A-T  
2  taxonomy:2697049  insdc:MN908947       insdc:MN908947:35-35-A-T  
3  taxonomy:2697049  insdc:MN908947       insdc:MN908947:36-36-C-T  
4  taxonomy:2697049  insdc:MN908947       insdc:MN908947:37-37-C-A

In [42]:
print('Number of variants:', variations.shape[0])

Number of variants: 86534


In [43]:
variations.to_csv(NEO4J_IMPORT / "01c-CNCBStrainVariant.csv", index=False)

In [44]:
t6 = time.time()
print('Time to parse variants:', t6-t5)

Time to parse variants: 1.3663761615753174


In [45]:
strains = pd.read_csv(NEO4J_IMPORT / "01c-CNCBStrainPre.csv", dtype=str)

In [46]:
strains.shape

(1905434, 15)

In [47]:
strains.head()

name                              accession  \
0  BetaCoV/Wuhan/HBCDC-HB-01/2019                        NMDC60013088-01   
1        hCoV-19/Thailand/74/2020  https://www.gisaid.org/EPI_ISL_403963   
2        hCoV-19/Thailand/61/2020  https://www.gisaid.org/EPI_ISL_403962   
3   BetaCoV/Wuhan/IVDC-HB-04/2020                        NMDC60013085-01   
4   BetaCoV/Wuhan/IVDC-HB-01/2019                        NMDC60013084-01   

                                          accessions  \
0  NMDC60013088-01;https://www.gisaid.org/EPI_ISL...   
1              https://www.gisaid.org/EPI_ISL_403963   
2              https://www.gisaid.org/EPI_ISL_403962   
3  NMDC60013085-01;https://www.gisaid.org/EPI_ISL...   
4  NMDC60013084-01;https://www.gisaid.org/EPI_ISL...   

                                gisaidId  source        taxonomyId  \
0  https://www.gisaid.org/EPI_ISL_402132    NMDC  taxonomy:2697049   
1  https://www.gisaid.org/EPI_ISL_403963  GISAID  taxonomy:2697049   
2  https://www.gisaid.org/EPI_ISL_403962  GISAID  taxonomy:2697049   
3  https://www.gisaid.org/EPI_ISL_402120    NMDC  taxonomy:2697049   
4  https://www.gisaid.org/EPI_ISL_402119    NMDC  taxonomy:2697049   

  hostTaxonomyId lineage sequenceLength sequenceQuality qualityAssessment  \
0  taxonomy:9606       B          29848            High         0/0/-/-/-   
1  taxonomy:9606       B          29859            High         0/0/-/-/-   
2  taxonomy:9606       B          29848            High         0/0/-/-/-   
3  taxonomy:9606       B          29896            High         0/0/-/-/-   
4  taxonomy:9606       B          29891            High         0/0/-/-/-   

  collectionDate                       location                  origLocation  \
0     2019-12-30                  China / Hubei                   China,Hubei   
1     2020-01-13  Thailand/ Nonthaburi Province  Thailand,Nonthaburi Province   
2     2020-01-08  Thailand/ Nonthaburi Province  Thailand,Nonthaburi Province   
3     2020-01-01          China / Hubei / Wuhan             China,Hubei,Wuhan   
4     2019-12-30          China / Hubei / Wuhan             China,Hubei,Wuhan   

                                      originatingLab  
0  Hubei Provincial Center for Disease Control an...  
1  Department of Medical Sciences, Ministry of Pu...  
2  Department of Medical Sciences, Ministry of Pu...  
3  National Institute for Viral Disease Control a...  
4  National Institute for Viral Disease Control a...

In [48]:
strains_acc = strains[['accession', 'accessions']].copy()
strains_acc['id'] = strains_acc['accessions'].str.split(';')
strains_acc = strains_acc.explode('id')
strains_acc['id'] = strains_acc['id'].str.strip()

In [49]:
strains_acc.shape

(2791885, 3)

In [50]:
strains_acc.head()

accession  \
0                        NMDC60013088-01   
0                        NMDC60013088-01   
1  https://www.gisaid.org/EPI_ISL_403963   
2  https://www.gisaid.org/EPI_ISL_403962   
3                        NMDC60013085-01   

                                          accessions  \
0  NMDC60013088-01;https://www.gisaid.org/EPI_ISL...   
0  NMDC60013088-01;https://www.gisaid.org/EPI_ISL...   
1              https://www.gisaid.org/EPI_ISL_403963   
2              https://www.gisaid.org/EPI_ISL_403962   
3  NMDC60013085-01;https://www.gisaid.org/EPI_ISL...   

                                      id  
0                        NMDC60013088-01  
0  https://www.gisaid.org/EPI_ISL_402132  
1  https://www.gisaid.org/EPI_ISL_403963  
2  https://www.gisaid.org/EPI_ISL_403962  
3                        NMDC60013085-01

In [51]:
strains_acc.drop_duplicates(inplace=True)
strains_acc.shape

(2791885, 3)

### Map variants to strains

In [52]:
var_ids = pd.DataFrame(variations['accession'].copy())
var_ids.rename(columns={'accession': 'id'}, inplace=True)
var_ids.drop_duplicates(inplace=True)

In [53]:
var_ids.head()

id
0   https://www.gisaid.org/EPI_ISL_423010
12  https://www.gisaid.org/EPI_ISL_423045
18  https://www.gisaid.org/EPI_ISL_415507
22  https://www.gisaid.org/EPI_ISL_414007
27  https://www.gisaid.org/EPI_ISL_419308

In [54]:
var_ids.shape

(5987, 1)

In [55]:
strains_map = strains_acc.merge(var_ids, on='id')

In [56]:
strains_map.head()

accession  \
0                        NMDC60013088-01   
1  https://www.gisaid.org/EPI_ISL_403963   
2  https://www.gisaid.org/EPI_ISL_403962   
3                        NMDC60013085-01   
4                        NMDC60013084-01   

                                          accessions  \
0  NMDC60013088-01;https://www.gisaid.org/EPI_ISL...   
1              https://www.gisaid.org/EPI_ISL_403963   
2              https://www.gisaid.org/EPI_ISL_403962   
3  NMDC60013085-01;https://www.gisaid.org/EPI_ISL...   
4  NMDC60013084-01;https://www.gisaid.org/EPI_ISL...   

                                      id  
0  https://www.gisaid.org/EPI_ISL_402132  
1  https://www.gisaid.org/EPI_ISL_403963  
2  https://www.gisaid.org/EPI_ISL_403962  
3  https://www.gisaid.org/EPI_ISL_402120  
4  https://www.gisaid.org/EPI_ISL_402119

In [57]:
strains_map.shape

(5983, 3)

In [58]:
strains_var = strains.merge(strains_map[['accession', 'id']], on='accession', how='outer')

In [59]:
strains_var['id'].fillna('', inplace=True)

In [60]:
strains_var['id'] = strains_var[['id', 'accession']].apply(lambda x: x[0] if x[0] != '' else x[1], axis=1)

In [61]:
strains_var.shape

(1905435, 16)

In [62]:
strains_var.head()

name                              accession  \
0  BetaCoV/Wuhan/HBCDC-HB-01/2019                        NMDC60013088-01   
1        hCoV-19/Thailand/74/2020  https://www.gisaid.org/EPI_ISL_403963   
2        hCoV-19/Thailand/61/2020  https://www.gisaid.org/EPI_ISL_403962   
3   BetaCoV/Wuhan/IVDC-HB-04/2020                        NMDC60013085-01   
4   BetaCoV/Wuhan/IVDC-HB-01/2019                        NMDC60013084-01   

                                          accessions  \
0  NMDC60013088-01;https://www.gisaid.org/EPI_ISL...   
1              https://www.gisaid.org/EPI_ISL_403963   
2              https://www.gisaid.org/EPI_ISL_403962   
3  NMDC60013085-01;https://www.gisaid.org/EPI_ISL...   
4  NMDC60013084-01;https://www.gisaid.org/EPI_ISL...   

                                gisaidId  source        taxonomyId  \
0  https://www.gisaid.org/EPI_ISL_402132    NMDC  taxonomy:2697049   
1  https://www.gisaid.org/EPI_ISL_403963  GISAID  taxonomy:2697049   
2  https://www.gisaid.org/EPI_ISL_403962  GISAID  taxonomy:2697049   
3  https://www.gisaid.org/EPI_ISL_402120    NMDC  taxonomy:2697049   
4  https://www.gisaid.org/EPI_ISL_402119    NMDC  taxonomy:2697049   

  hostTaxonomyId lineage sequenceLength sequenceQuality qualityAssessment  \
0  taxonomy:9606       B          29848            High         0/0/-/-/-   
1  taxonomy:9606       B          29859            High         0/0/-/-/-   
2  taxonomy:9606       B          29848            High         0/0/-/-/-   
3  taxonomy:9606       B          29896            High         0/0/-/-/-   
4  taxonomy:9606       B          29891            High         0/0/-/-/-   

  collectionDate                       location                  origLocation  \
0     2019-12-30                  China / Hubei                   China,Hubei   
1     2020-01-13  Thailand/ Nonthaburi Province  Thailand,Nonthaburi Province   
2     2020-01-08  Thailand/ Nonthaburi Province  Thailand,Nonthaburi Province   
3     2020-01-01          China / Hubei / Wuhan             China,Hubei,Wuhan   
4     2019-12-30          China / Hubei / Wuhan             China,Hubei,Wuhan   

                                      originatingLab  \
0  Hubei Provincial Center for Disease Control an...   
1  Department of Medical Sciences, Ministry of Pu...   
2  Department of Medical Sciences, Ministry of Pu...   
3  National Institute for Viral Disease Control a...   
4  National Institute for Viral Disease Control a...   

                                      id  
0  https://www.gisaid.org/EPI_ISL_402132  
1  https://www.gisaid.org/EPI_ISL_403963  
2  https://www.gisaid.org/EPI_ISL_403962  
3  https://www.gisaid.org/EPI_ISL_402120  
4  https://www.gisaid.org/EPI_ISL_402119

In [63]:
strains_var.to_csv(NEO4J_IMPORT / "01c-CNCBStrain.csv", index=False)

In [64]:
t7 = time.time()
print('Time to map variants to strains:', t7-t6)

Time to map variants to strains: 71.6434257030487


### Save unique variants
To find all unique variants, drop the strain names and accession numbers. The remaining fields are variant specific.

In [65]:
print('Number of unique variants:', len(variations['id'].unique()))

Number of unique variants: 12047


In [66]:
variations.drop(columns=['name', 'accession'], inplace=True)

In [67]:
variations.drop_duplicates(inplace=True)

In [68]:
# TODO there are a few cases with the same id but different variant data ???
print('Number of unique variants:', variations.shape[0])

Number of unique variants: 12127


In [69]:
variations.head()

variantType  start  end       ref alt  variantConsequence proteinVariant  \
0    Deletion      1    7  AATTAAAG   -  intergenic_variant                  
1         SNP     34   34         A   T  intergenic_variant                  
2         SNP     35   35         A   T  intergenic_variant                  
3         SNP     36   36         C   T  intergenic_variant                  
4         SNP     37   37         C   A  intergenic_variant                  

  geneVariant distance proteinPosition proteinAccession        taxonomyId  \
0                                                        taxonomy:2697049   
1                                                        taxonomy:2697049   
2                                                        taxonomy:2697049   
3                                                        taxonomy:2697049   
4                                                        taxonomy:2697049   

  referenceGenome                             id  
0  insdc:MN908947  insdc:MN908947:1-7-AATTAAAG--  
1  insdc:MN908947       insdc:MN908947:34-34-A-T  
2  insdc:MN908947       insdc:MN908947:35-35-A-T  
3  insdc:MN908947       insdc:MN908947:36-36-C-T  
4  insdc:MN908947       insdc:MN908947:37-37-C-A

In [70]:
variations.to_csv(NEO4J_IMPORT / "01c-CNCBVariant.csv", index=False)